In [1]:
import pandas as pd

In [2]:
import requests

url = 'https://raw.githubusercontent.com/ckirby19/league-of-legends-player-profiles/api_data/json_data_test.json'
response = requests.get(url)
d = response.json()

In [3]:
frames = d['info']['frames']

In [4]:
events = [event for frame in frames for event in frame['events']]

event_df = pd.DataFrame(events)

In [6]:
display(event_df)

,realTimestamp,timestamp,type,itemId,participantId,afterId,beforeId,goldGain,levelUpType,skillSlot,...,laneType,teamId,killerTeamId,monsterSubType,monsterType,buildingType,towerType,actualStartTime,gameId,winningTeam
0,1.712859e+12,0,PAUSE_END,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2036,ITEM_PURCHASED,3865.0,10.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2036,ITEM_PURCHASED,2003.0,10.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,2036,ITEM_PURCHASED,2003.0,10.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,2336,ITEM_PURCHASED,1102.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951,NaN,1727532,CHAMPION_SPECIAL_KILL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
952,NaN,1727532,LEVEL_UP,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
953,NaN,1727797,CHAMPION_KILL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
954,NaN,1727797,CHAMPION_SPECIAL_KILL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
